## College Football Machine Learning 2008-2013

In [3]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Read the CSV 

In [6]:
# Load the data
from google.colab import files
uploaded = files.upload()

file_name='goldcopyhalfstats.csv'
#df = pd.read_csv(file_name, skiprows=0)[:-2]
df=pd.read_csv(file_name)

Saving goldcopyhalfstats.csv to goldcopyhalfstats (1).csv


In [7]:
df.head()

,year,game_code,a_team_name,a_name_conference,a_team_code,a_result,a_points,b_team_code,team_name,name_conference,b_result,b_points,half_leader,a_rush,b_rush,a_offense_rush_yards,b_defense_rush_yards,a_defense_rush_yards,b_offense_rush_yards,a_offense_pass_yards,b_defense_pass_yards,a_defense_pass_yards,b_offense_pass_yards,a_pass,b_pass,a_incomplete_pass,b_incomplete_pass,a_penalty,b_penalty,a_penalty_yards,b_penalty_yards,half_diff_score,half_diff_score_abs,hometeam,b_home,visit_date,stadium_code,stadium_name,site,city,state,capacity,surface,surfaceint,year_opened
0,2013,2.050000e+12,New Mexico State ...,Independent ...,472,1,21,2,Abilene Christian ...,Ind ...,0,22,BTEAM,20,16,100,100,87,87,169,169,159,159,18,21,3,5,2,2,10,15,-1,1,A,0,10/26/2013,88,Aggie Stadium ...,TEAM,Las Cruces ...,NM,40000,Grass,1,1978
1,2010,5.000000e+12,Akron ...,Mid-American Conference ...,5,0,7,47,Ball State ...,Mid-American Conference ...,1,13,BTEAM,18,18,32,32,110,110,71,71,42,42,15,14,8,5,1,1,5,0,-6,6,B,1,11/6/2010,48,Scheumann Stadium ...,TEAM,Munice ...,IN,22500,FieldTurf,0,1967
2,2009,5.010000e+12,Bowling Green ...,Mid-American Conference ...,71,1,9,5,Akron ...,Mid-American Conference ...,0,10,BTEAM,13,21,-81,-81,85,85,166,166,60,60,18,10,6,3,3,0,20,0,-1,1,A,0,11/20/2009,96,Doyt Perry Stadium ...,TEAM,Bowling Green ...,OH,28599,FieldTurf,0,1966
3,2013,5.010000e+12,Akron ...,Mid-American Conference ...,5,0,14,71,Bowling Green ...,Mid-American Conference ...,1,10,ATEAM,15,21,61,61,82,82,137,137,73,73,23,13,8,6,2,1,23,5,4,4,B,1,9/28/2013,96,Doyt Perry Stadium ...,TEAM,Bowling Green ...,OH,28599,FieldTurf,0,1966
4,2009,5.010000e+12,Akron ...,Mid-American Conference ...,5,0,7,86,Buffalo ...,Mid-American Conference ...,1,7,TIED,18,18,-13,-13,43,43,0,0,144,144,11,21,3,9,1,4,15,15,0,0,B,1,10/17/2009,91,UB Stadium ...,TEAM,Buffalo ...,NY,31000,Sportexe Momentum,0,1993


#Filter Data

In [10]:
# Throw out extraneous columns
df2=df.copy()
df2["result"]=df2['a_result']
df2=df2.drop(['a_result', 'b_result'], axis=1)

df2=df2.drop(['a_defense_rush_yards','b_defense_rush_yards','a_defense_pass_yards','b_defense_pass_yards'],axis=1)

df2.head()

,year,game_code,a_team_name,a_name_conference,a_team_code,a_points,b_team_code,team_name,name_conference,b_points,half_leader,a_rush,b_rush,a_offense_rush_yards,b_offense_rush_yards,a_offense_pass_yards,b_offense_pass_yards,a_pass,b_pass,a_incomplete_pass,b_incomplete_pass,a_penalty,b_penalty,a_penalty_yards,b_penalty_yards,half_diff_score,half_diff_score_abs,hometeam,b_home,visit_date,stadium_code,stadium_name,site,city,state,capacity,surface,surfaceint,year_opened,result
0,2013,2.050000e+12,New Mexico State ...,Independent ...,472,21,2,Abilene Christian ...,Ind ...,22,BTEAM,20,16,100,87,169,159,18,21,3,5,2,2,10,15,-1,1,A,0,10/26/2013,88,Aggie Stadium ...,TEAM,Las Cruces ...,NM,40000,Grass,1,1978,1
1,2010,5.000000e+12,Akron ...,Mid-American Conference ...,5,7,47,Ball State ...,Mid-American Conference ...,13,BTEAM,18,18,32,110,71,42,15,14,8,5,1,1,5,0,-6,6,B,1,11/6/2010,48,Scheumann Stadium ...,TEAM,Munice ...,IN,22500,FieldTurf,0,1967,0
2,2009,5.010000e+12,Bowling Green ...,Mid-American Conference ...,71,9,5,Akron ...,Mid-American Conference ...,10,BTEAM,13,21,-81,85,166,60,18,10,6,3,3,0,20,0,-1,1,A,0,11/20/2009,96,Doyt Perry Stadium ...,TEAM,Bowling Green ...,OH,28599,FieldTurf,0,1966,1
3,2013,5.010000e+12,Akron ...,Mid-American Conference ...,5,14,71,Bowling Green ...,Mid-American Conference ...,10,ATEAM,15,21,61,82,137,73,23,13,8,6,2,1,23,5,4,4,B,1,9/28/2013,96,Doyt Perry Stadium ...,TEAM,Bowling Green ...,OH,28599,FieldTurf,0,1966,0
4,2009,5.010000e+12,Akron ...,Mid-American Conference ...,5,7,86,Buffalo ...,Mid-American Conference ...,7,TIED,18,18,-13,43,0,144,11,21,3,9,1,4,15,15,0,0,B,1,10/17/2009,91,UB Stadium ...,TEAM,Buffalo ...,NY,31000,Sportexe Momentum,0,1993,0


In [13]:
df_filter=df2.copy()

# Throw out "blowout" games
blowoutThresh=14
df_filter=df_filter[df_filter['half_diff_score_abs'] <= blowoutThresh]

# reset index
df_filter.reset_index(inplace=True, drop=True)

print(len(df2))
print(len(df_filter))
df_filter.head(5)

3242
3242


,year,game_code,a_team_name,a_name_conference,a_team_code,a_points,b_team_code,team_name,name_conference,b_points,half_leader,a_rush,b_rush,a_offense_rush_yards,b_offense_rush_yards,a_offense_pass_yards,b_offense_pass_yards,a_pass,b_pass,a_incomplete_pass,b_incomplete_pass,a_penalty,b_penalty,a_penalty_yards,b_penalty_yards,half_diff_score,half_diff_score_abs,hometeam,b_home,visit_date,stadium_code,stadium_name,site,city,state,capacity,surface,surfaceint,year_opened,result
0,2013,2.050000e+12,New Mexico State ...,Independent ...,472,21,2,Abilene Christian ...,Ind ...,22,BTEAM,20,16,100,87,169,159,18,21,3,5,2,2,10,15,-1,1,A,0,10/26/2013,88,Aggie Stadium ...,TEAM,Las Cruces ...,NM,40000,Grass,1,1978,1
1,2010,5.000000e+12,Akron ...,Mid-American Conference ...,5,7,47,Ball State ...,Mid-American Conference ...,13,BTEAM,18,18,32,110,71,42,15,14,8,5,1,1,5,0,-6,6,B,1,11/6/2010,48,Scheumann Stadium ...,TEAM,Munice ...,IN,22500,FieldTurf,0,1967,0
2,2009,5.010000e+12,Bowling Green ...,Mid-American Conference ...,71,9,5,Akron ...,Mid-American Conference ...,10,BTEAM,13,21,-81,85,166,60,18,10,6,3,3,0,20,0,-1,1,A,0,11/20/2009,96,Doyt Perry Stadium ...,TEAM,Bowling Green ...,OH,28599,FieldTurf,0,1966,1
3,2013,5.010000e+12,Akron ...,Mid-American Conference ...,5,14,71,Bowling Green ...,Mid-American Conference ...,10,ATEAM,15,21,61,82,137,73,23,13,8,6,2,1,23,5,4,4,B,1,9/28/2013,96,Doyt Perry Stadium ...,TEAM,Bowling Green ...,OH,28599,FieldTurf,0,1966,0
4,2009,5.010000e+12,Akron ...,Mid-American Conference ...,5,7,86,Buffalo ...,Mid-American Conference ...,7,TIED,18,18,-13,43,0,144,11,21,3,9,1,4,15,15,0,0,B,1,10/17/2009,91,UB Stadium ...,TEAM,Buffalo ...,NY,31000,Sportexe Momentum,0,1993,0


In [14]:
df_filter.columns

Index(['year', 'game_code', 'a_team_name', 'a_name_conference', 'a_team_code',
       'a_points', 'b_team_code', 'team_name', 'name_conference', 'b_points',
       'half_leader', 'a_rush', 'b_rush', 'a_offense_rush_yards',
       'b_offense_rush_yards', 'a_offense_pass_yards', 'b_offense_pass_yards',
       'a_pass', 'b_pass', 'a_incomplete_pass', 'b_incomplete_pass',
       'a_penalty', 'b_penalty', 'a_penalty_yards', 'b_penalty_yards',
       'half_diff_score', 'half_diff_score_abs', 'hometeam', 'b_home',
       'visit_date', 'stadium_code', 'stadium_name', 'site', 'city', 'state',
       'capacity', 'surface', 'surfaceint', 'year_opened', 'result'],
      dtype='object')

# Split the Data into Training and Testing and Scale

In [18]:
# Create our features
X = df_filter.copy()
#X=X.drop({'year', 'game_code', 'a_team_name', 'a_name_conference', 'a_team_code',
#          'b_team_code', 'team_name', 'name_conference','half_leader',
#          'half_diff_score', 'half_diff_score_abs', 'visit_date', 'stadium_code',
#          'stadium_name', 'site', 'city', 'state', 'capacity',
#          'year_opened', 'result'},axis=1)

X=X.drop({'year', 'game_code', 'a_team_name', 'a_name_conference', 'a_team_code',
       'b_team_code', 'team_name', 'name_conference',
       'half_leader', 'half_diff_score', 'half_diff_score_abs', 'hometeam',
       'visit_date', 'stadium_code', 'stadium_name', 'site', 'city', 'state',
       'surface','year_opened', 'result'},axis=1)
print(X.columns)

# Need to convert columns with strings to numeric
# Get column names that are non-numeric
column_names=X.select_dtypes(exclude=[np.number]).columns
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for columnName in column_names:
  X[columnName]=le.fit_transform(X[columnName])

# Create our target
y = df_filter['result'].copy()

X.head()

Index(['a_points', 'b_points', 'a_rush', 'b_rush', 'a_offense_rush_yards',
       'b_offense_rush_yards', 'a_offense_pass_yards', 'b_offense_pass_yards',
       'a_pass', 'b_pass', 'a_incomplete_pass', 'b_incomplete_pass',
       'a_penalty', 'b_penalty', 'a_penalty_yards', 'b_penalty_yards',
       'b_home', 'capacity', 'surfaceint'],
      dtype='object')


,a_points,b_points,a_rush,b_rush,a_offense_rush_yards,b_offense_rush_yards,a_offense_pass_yards,b_offense_pass_yards,a_pass,b_pass,a_incomplete_pass,b_incomplete_pass,a_penalty,b_penalty,a_penalty_yards,b_penalty_yards,b_home,capacity,surfaceint
0,21,22,20,16,100,87,169,159,18,21,3,5,2,2,10,15,0,40000,1
1,7,13,18,18,32,110,71,42,15,14,8,5,1,1,5,0,1,22500,0
2,9,10,13,21,-81,85,166,60,18,10,6,3,3,0,20,0,0,28599,0
3,14,10,15,21,61,82,137,73,23,13,8,6,2,1,23,5,1,28599,0
4,7,7,18,18,-13,43,0,144,11,21,3,9,1,4,15,15,1,31000,0


In [20]:
# Check the balance of our target values
print(len(X))
abs(X['a_points']-X['b_points']).value_counts()

# We have a lot of games with a 7-point spread at half compared to the others (makes sense)
# May want to look at stratifying?  So that our model predicts well at any spread

3242


7     516
3     397
14    375
10    343
0     330
4     319
11    234
1     145
6     138
13    137
8     106
2      59
9      58
5      52
12     33
dtype: int64

# Sampling Methods and Scale


In [21]:

from sklearn.model_selection import train_test_split

# stratify by point spread
stratify_values=abs(df_filter['a_points']-df_filter['b_points'])
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    stratify=stratify_values, 
                                                    test_size=0.25)

# Scale the Data
#from sklearn.preprocessing import StandardScaler
#from sklearn.preprocessing import MinMaxScaler

#scaler = StandardScaler()
#scaler=MinMaxScaler()
#scaler=StandardScaler()
#X_scaler = scaler.fit(X_train)

#X_train_scaled = X_scaler.transform(X_train)
#X_test_scaled = X_scaler.transform(X_test)
#X_train_scaled = X_train
#X_test_scaled = X_test


# Models

## Import Dependencies

In [22]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import balanced_accuracy_score
from imblearn.metrics import classification_report_imbalanced

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler


## Logistic Regression

In [23]:
classifier = LogisticRegression(solver='lbfgs',
                                max_iter=1500,
                                random_state=42)

#result=classifier.fit(X_train_scaled, y_train)
result=classifier.fit(X_train, y_train)

In [24]:
ml_df=pd.DataFrame(columns=X.columns.tolist())

log_coefs=list(zip(X.columns.tolist(),
                   result.coef_.tolist()[0]))
log_coefs.sort(key = lambda x:abs(x[1]),reverse=True)
print(log_coefs)

[('b_points', -0.1628593885833281), ('a_points', 0.1437162187022262), ('b_incomplete_pass', 0.013847220173410437), ('a_rush', -0.00998635193148035), ('a_incomplete_pass', -0.0067324575001523465), ('b_home', -0.005895922830335066), ('a_pass', -0.0049379516515968645), ('b_pass', -0.004372119185497701), ('a_offense_pass_yards', 0.004106326141254591), ('a_offense_rush_yards', 0.0030381338419710768), ('a_penalty_yards', -0.0024299254988259518), ('b_rush', -0.002404177593787222), ('b_penalty_yards', 0.0015378833848268179), ('b_penalty', -0.0010744302394365146), ('a_penalty', 0.0009272124413880311), ('surfaceint', 0.0008527012885825396), ('b_offense_rush_yards', 0.00019286332933831572), ('b_offense_pass_yards', -0.0001450091125335476), ('capacity', 1.0117671579436158e-06)]


In [25]:
y_pred = classifier.predict(X_test)
results = pd.DataFrame({"Prediction": y_pred, "Actual": y_test}).reset_index(drop=True)
print(classifier.get_params())
results.head(10)

{'C': 1.0, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 1500, 'multi_class': 'auto', 'n_jobs': None, 'penalty': 'l2', 'random_state': 42, 'solver': 'lbfgs', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}


,Prediction,Actual
0,1,0
1,0,1
2,1,1
3,1,1
4,1,0
5,1,1
6,0,1
7,1,0
8,1,1
9,0,0


In [26]:
logRegAcc=balanced_accuracy_score(y_test, y_pred)
noDroplogRegAcc=logRegAcc
print('balanced accuracy score = ',logRegAcc)

# Display the confusion matrix
# Calculating the confusion matrix.
cm = confusion_matrix(y_test, y_pred)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual B", "Actual A"], columns=["Predicted B", "Predicted A"])

print("")
print(cm_df)

# Print the imbalanced classification report
print("")
print(classification_report_imbalanced(y_test, y_pred))

balanced accuracy score =  0.7553103877406603

          Predicted B  Predicted A
Actual B          277          106
Actual A           91          337

                   pre       rec       spe        f1       geo       iba       sup

          0       0.75      0.72      0.79      0.74      0.75      0.57       383
          1       0.76      0.79      0.72      0.77      0.75      0.57       428

avg / total       0.76      0.76      0.75      0.76      0.75      0.57       811



# Sensitivity Tests

In [27]:
# We will drop each column and see what the balanced score gives us
X.columns
print('No Drop Balanced Accuracy = ',round(noDroplogRegAcc,2))
for column in X.columns:
  #Creata new Dataframe with dropped column
  Xtest=X.drop(column,axis=1).copy()

  # Gotta train, test, split using same random seed
  # stratify by point spread
  stratify_values=abs(df_filter['a_points']-df_filter['b_points'])
  X_train, X_test, y_train, y_test = train_test_split(Xtest, y,
                                                    stratify=stratify_values, 
                                                    test_size=0.3)

  # Scale the Data
  #scaler = StandardScaler()
  #scaler=MinMaxScaler()
  #scaler=StandardScaler()
  #X_scaler = scaler.fit(X_train)

  #X_train_scaled = X_scaler.transform(X_train)
  #X_test_scaled = X_scaler.transform(X_test)

  # ML with logistic Regression
  classifier = LogisticRegression(solver='lbfgs',
                                max_iter=2500,
                                random_state=42)

  #result=classifier.fit(X_train_scaled, y_train)
  result=classifier.fit(X_train, y_train)
  y_pred = classifier.predict(X_test)
  logResAcc=balanced_accuracy_score(y_test, y_pred)

  print('Dropped ',column,', balanced acc. = ',round(logResAcc,2))


No Drop Balanced Accuracy =  0.76
Dropped  a_points , balanced acc. =  0.69
Dropped  b_points , balanced acc. =  0.67
Dropped  a_rush , balanced acc. =  0.72
Dropped  b_rush , balanced acc. =  0.71
Dropped  a_offense_rush_yards , balanced acc. =  0.73
Dropped  b_offense_rush_yards , balanced acc. =  0.71
Dropped  a_offense_pass_yards , balanced acc. =  0.71
Dropped  b_offense_pass_yards , balanced acc. =  0.73
Dropped  a_pass , balanced acc. =  0.72
Dropped  b_pass , balanced acc. =  0.73
Dropped  a_incomplete_pass , balanced acc. =  0.72
Dropped  b_incomplete_pass , balanced acc. =  0.73
Dropped  a_penalty , balanced acc. =  0.74
Dropped  b_penalty , balanced acc. =  0.73
Dropped  a_penalty_yards , balanced acc. =  0.73
Dropped  b_penalty_yards , balanced acc. =  0.74
Dropped  b_home , balanced acc. =  0.74
Dropped  capacity , balanced acc. =  0.74
Dropped  surfaceint , balanced acc. =  0.73


# Drop unneeded features

In [29]:
Xnew=X.drop({'a_pass','b_pass',
             'a_rush','b_rush',
             'a_penalty','b_penalty',
             'capacity','surfaceint'
            },axis=1).copy()


# Gotta train, test, split using same random seed
# stratify by point spread
stratify_values=abs(df_filter['a_points']-df_filter['b_points'])
X_train, X_test, y_train, y_test = train_test_split(Xnew, y,
                                                    stratify=stratify_values, 
                                                    test_size=0.25)

# Scale the Data
#scaler = StandardScaler()
#scaler=MinMaxScaler()
#scaler=StandardScaler()
#X_scaler = scaler.fit(X_train)

#X_train_scaled = X_scaler.transform(X_train)
#X_test_scaled = X_scaler.transform(X_test)

# ML with logistic Regression
classifier = LogisticRegression(solver='lbfgs',
                                max_iter=2500,
                                random_state=42)

#result=classifier.fit(X_train_scaled, y_train)
result=classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
logResAcc=balanced_accuracy_score(y_test, y_pred)
print('Old Balanced Accuracy = ',round(noDroplogRegAcc,2))
print('New Balanced Accuracy = ',round(logResAcc,2))

Old Balanced Accuracy =  0.76
New Balanced Accuracy =  0.74


In [30]:
# Create Dataframe with fit coefficients and print to file
coefficients=result.coef_.tolist()[0]
coef_df=pd.DataFrame()
i=0;
for feature in Xnew.columns:
  coef_df[feature]=[coefficients[i]]
  i=i+1
coef_df

,a_points,b_points,a_offense_rush_yards,b_offense_rush_yards,a_offense_pass_yards,b_offense_pass_yards,a_incomplete_pass,b_incomplete_pass,a_penalty_yards,b_penalty_yards,b_home
0,0.16084,-0.162158,0.001802,-0.000725,0.002275,-0.00146,0.022923,-0.005617,-0.002272,0.009288,-0.433446


In [31]:
# Create Dataframe with fit coefficients and print to file
coefficients=result.coef_.tolist()[0]
coef_df=pd.DataFrame()
i=0;
for feature in Xnew.columns:
  coef_df[feature]=[coefficients[i]]
  i=i+1
print(coef_df)

coef_df.to_csv('coefs.csv',index=False)
from google.colab import files
files.download("coefs.csv")

   a_points  b_points  ...  b_penalty_yards    b_home
0   0.16084 -0.162158  ...         0.009288 -0.433446

[1 rows x 11 columns]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [32]:
y_pred = classifier.predict(X_test)
results = pd.DataFrame({"Prediction": y_pred, "Actual": y_test}).reset_index(drop=True)
print(classifier.get_params())
results.head(10)

{'C': 1.0, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 2500, 'multi_class': 'auto', 'n_jobs': None, 'penalty': 'l2', 'random_state': 42, 'solver': 'lbfgs', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}


,Prediction,Actual
0,1,1
1,0,1
2,0,0
3,0,0
4,0,0
5,0,0
6,0,0
7,0,0
8,0,0
9,0,0


In [33]:
logRegAcc=balanced_accuracy_score(y_test, y_pred)
noDroplogRegAcc=logRegAcc
print('balanced accuracy score = ',logRegAcc)

# Display the confusion matrix
# Calculating the confusion matrix.
cm = confusion_matrix(y_test, y_pred)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual B", "Actual A"], columns=["Predicted B", "Predicted A"])

print("")
print(cm_df)

# Print the imbalanced classification report
print("")
print(classification_report_imbalanced(y_test, y_pred))

balanced accuracy score =  0.7400017028729899

          Predicted B  Predicted A
Actual B          282          125
Actual A           86          318

                   pre       rec       spe        f1       geo       iba       sup

          0       0.77      0.69      0.79      0.73      0.74      0.54       407
          1       0.72      0.79      0.69      0.75      0.74      0.55       404

avg / total       0.74      0.74      0.74      0.74      0.74      0.55       811



# Average Games (for plotting purposes)

In [34]:
X.mean()

a_points                   12.687847
b_points                   12.836212
a_rush                     17.741518
b_rush                     18.648365
a_offense_rush_yards       71.197101
b_offense_rush_yards       72.465453
a_offense_pass_yards      101.956817
b_offense_pass_yards      115.425355
a_pass                     15.217458
b_pass                     17.085133
a_incomplete_pass           5.741209
b_incomplete_pass           6.449722
a_penalty                   2.316780
b_penalty                   2.307218
a_penalty_yards            17.053054
b_penalty_yards            17.133868
b_home                      0.581740
capacity                54708.426280
surfaceint                  0.429365
dtype: float64